In [1]:
import bgen_reader
import pandas as pd

# Load data

In [2]:
filedir = '/vol/bmd/meliao/data/haplotype/hap/'
chr_num = '22'

In [3]:
bgen = bgen_reader.read_bgen(
    f'{filedir}/ukb_hap_chr{chr_num}_v2.bgen', 
    samples_filepath = '/vol/bmd/meliao/data/haplotype/link_files/ukb1952_v2_s487398.sample'
)

/vol/bmd/yanyul/miniconda3/envs/haplotype_po/lib/python3.7/site-packages/bgen_reader/_reader.py:135: UserWarning: You don't have permission to write to `/vol/bmd/meliao/data/haplotype/hap/ukb_hap_chr22_v2.bgen.metadata`.
This might prevent speeding-up the reading process in future runs.

  warnings.warn(_metafile_nowrite_dir.format(filepath=fp), UserWarning)


We will create the metafile `/tmp/tmps173syr2/%vol%bmd%meliao%data%haplotype%hapukb_hap_chr22_v2.bgen.metadata`. This file will speed up further
reads and only need to be created once. So, please, bear with me.
Sample IDs are read from /vol/bmd/meliao/data/haplotype/link_files/ukb1952_v2_s487398.sample.


Mapping variants: 100%|██████████| 10911/10911 [00:00<00:00, 67679.48it/s]


In [4]:
gwas = pd.read_csv(
    '/vol/bmd/yanyul/UKB/haplotype_imputation/20002_1065.gwas.imputed_v3.both_sexes.tsv.bgz', 
    header = 0, 
    sep = '\t', 
    compression = 'gzip'
)

In [16]:
snp_map = pd.read_csv(
    '/vol/bmd/yanyul/UKB/haplotype_imputation/snp_map_for_neale_lab_gwas.tsv.gz',
    sep='\t',
    header=0,
    compression='gzip',
    dtype={3:'str'}
)

# Workflow

1. **In GWAS**, extract variants that occur in my genotype.
2. **In GWAS**, perform LD clumping.
3. Calculate PRS.

In [17]:
snp_map = snp_map[ (snp_map['assigned_id'] != 'not_shown') & (snp_map['assigned_id'] != 'ambiguous') ]

In [20]:
# extract variants from GWAS
gwas_sub = gwas[ gwas['variant'].isin(snp_map['assigned_id']) ]

In [21]:
# gwas_sub.to_csv()

,variant,minor_allele,minor_AF,expected_case_minor_AC,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval
23,1:754105:C:T,T,0.036358,6803.38,False,361141,26261.0,6774.0,-0.000939,0.002727,-0.344463,0.730499
32,1:756604:A:G,A,0.129866,24300.60,False,361141,628482.0,162876.0,0.000781,0.001491,0.524009,0.600273
55,1:768448:G:A,A,0.105920,19819.70,False,361141,76504.0,19920.0,0.001463,0.001626,0.899687,0.368287
77,1:779322:A:G,G,0.127848,23922.80,False,361141,92342.0,23900.0,-0.000413,0.001500,-0.275184,0.783175
124,1:801536:T:G,G,0.015058,2817.62,False,361141,10876.0,2813.0,-0.000808,0.004117,-0.196378,0.844315
...,...,...,...,...,...,...,...,...,...,...,...,...
13364106,22:51177257:C:T,T,0.037381,6994.83,False,361141,27000.0,7086.0,0.002953,0.002644,1.117070,0.263965
13364151,22:51183255:A:G,A,0.334182,62532.20,False,361141,480908.0,124984.0,0.002137,0.001062,2.013180,0.044097
13364161,22:51185848:G:A,G,0.050516,9452.61,False,361141,685795.0,177679.0,0.001040,0.002284,0.455429,0.648801
13364198,22:51193629:A:G,G,0.076289,14275.20,False,361141,55102.0,14323.0,0.001454,0.001885,0.771293,0.440534


In [22]:
snp_map

,chrom,pos,allele_ids,id,rsid,assigned_id
1,1,754105,"C,T",1,rs12184325,1:754105:C:T
2,1,756604,"A,G",1,rs3131962,1:756604:A:G
5,1,768448,"G,A",1,rs12562034,1:768448:G:A
6,1,779322,"A,G",1,rs4040617,1:779322:A:G
7,1,801536,"T,G",1,rs79373928,1:801536:T:G
...,...,...,...,...,...,...
658714,22,51177257,"T,C",rs73174437,rs73174437,22:51177257:C:T
658715,22,51183255,"A,G",rs5771002,rs5771002,22:51183255:A:G
658716,22,51185848,"G,A",rs3865764,rs3865764,22:51185848:G:A
658717,22,51193629,"G,A",rs142680588,rs142680588,22:51193629:A:G
